# Using crack submodels in PyBaMM
In this notebook we show how to use the crack submodel with battery DFN or SPM models. To see all of the models and submodels available in PyBaMM, please take a look at the documentation [here](https://pybamm.readthedocs.io/en/latest/source/api/models/index.html).

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

Note: you may need to restart the kernel to use updated packages.


Then we load the DFN, SPMe or SPM, by choosing one and commenting the others. 

When you load a model in PyBaMM it builds by default. Building the model sets all of the model variables and sets up any variables which are coupled between different submodels: this is the process which couples the submodels together and allows one submodel to access variables from another. 

In [2]:
model = pybamm.lithium_ion.DFN(
    options = {
        "particle": "Fickian diffusion",  
        "particle mechanics": "swelling and cracking", # other options are "none", "swelling only"
    }
)

Load the parameter set Ai2020 which contains mechanical parameters. Other sets may not contain mechanical parameters should add them manually. 

In [3]:
param = pybamm.ParameterValues("Ai2020")
## It can update the speed of crack propagation using the commands below:
# param.update({"Negative electrode Cracking rate":3.9e-20*10})


We can get the default parameters for the model and update them with the parameters required by the cracking model. Eventually, we would like these to be added to their own chemistry (you might need to adjust the path to the parameters file to your system).
Now the model can be processed and solved in the usual way, and we still have access to model defaults such as the default geometry and default spatial methods

In [4]:
sim = pybamm.Simulation(
    model,
    parameter_values=param,
    solver=pybamm.CasadiSolver(dt_max=600),
)
solution = sim.solve(t_eval=[0, 3600], inputs={"C-rate": 1})
# plot
quick_plot = pybamm.QuickPlot(solution)
quick_plot.dynamic_plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

Plot the results as required.

In [5]:
# extract voltage
E_n = param["Negative electrode Young's modulus [Pa]"]
stress_t_n_surf = solution["Negative particle surface tangential stress [Pa]"]
x = solution["x [m]"].entries[0:19, 0]
c_s_n = solution['Negative particle concentration']
r_n = solution["r_n [m]"].entries[:, 0, 0]

# plot
def plot_concentrations(t):
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4 ,figsize=(20,4))
    ax1.plot(x, stress_t_n_surf(t=t,x=x) / E_n)
    ax1.set_xlabel(r'$x_n$ [m]')
    ax1.set_ylabel('$\sigma_t/E_n$')
    
    plot_c_n, = ax2.plot(r_n, c_s_n(r=r_n,t=t,x=x[0]))  # can evaluate at arbitrary x (single representative particle)
    ax2.set_ylabel('Negative particle concentration')
    ax2.set_xlabel(r'$r_n$ [m]')
    ax2.set_ylim(0, 1)
    ax2.set_title('Close to current collector')
    ax2.grid()
    
    plot_c_n, = ax3.plot(r_n, c_s_n(r=r_n,t=t,x=x[10]))  # can evaluate at arbitrary x (single representative particle)
    ax3.set_ylabel('Negative particle concentration')
    ax3.set_xlabel(r'$r_n$ [m]')
    ax3.set_ylim(0, 1)  
    ax3.set_title('In the middle')
    ax3.grid()

    plot_c_n, = ax4.plot(r_n, c_s_n(r=r_n,t=t,x=x[-1]))  # can evaluate at arbitrary x (single representative particle)
    ax4.set_ylabel('Negative particle concentration')
    ax4.set_xlabel(r'$r_n$ [m]')
    ax4.set_ylim(0, 1)  
    ax4.set_title('Close to separator')
    ax4.grid()
    plt.show()
    
import ipywidgets as widgets
widgets.interact(plot_concentrations, t=widgets.FloatSlider(min=0,max=3600,step=10,value=0));

interactive(children=(FloatSlider(value=0.0, description='t', max=3600.0, step=10.0), Output()), _dom_classes=…

Plot results using the default functions

In [6]:
label = ["Crack model"]
output_variables = [
    "Negative particle crack length [m]", 
    "Positive particle crack length [m]",
    "X-averaged negative particle crack length [m]",
    "X-averaged positive particle crack length [m]"
]
quick_plot = pybamm.QuickPlot(solution, output_variables, label,variable_limits='tight')
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## References

The relevant papers for this notebook are:

In [7]:
pybamm.print_citations()

[1] Weilong Ai, Ludwig Kraft, Johannes Sturm, Andreas Jossen, and Billy Wu. Electrochemical thermal-mechanical modelling of stress inhomogeneity in lithium-ion pouch cells. Journal of The Electrochemical Society, 167(1):013512, 2019. doi:10.1149/2.0122001JES.
[2] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[3] Rutooj Deshpande, Mark Verbrugge, Yang-Tse Cheng, John Wang, and Ping Liu. Battery cycle life prediction with coupled chemical degradation and fatigue mechanics. Journal of the Electrochemical Society, 159(10):A1730, 2012. doi:10.1149/2.049210jes.
[4] Marc Doyle, Thomas F. Fuller, and John Newman. Modeling of galvanostatic charge and discharge of the lithium/polymer/insertion cell. Journal of the Electrochemical society, 140(6):1526–1533, 1993. doi:10.1149/1.2221597.
[5] Charles